In [1]:
print("Hello World")

Hello World


In [2]:
import pandas as pd
import pyodbc

In [3]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [4]:
# Write sql query
qry = "SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_award;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\1863516591.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [5]:
df['award_reference'].isna().sum()

440

In [6]:
df['award_reference'] = df['award_reference'].replace('Unknown', None)

In [7]:
df[df['award_reference'].isna()]

print(df['award_reference'].isna().sum()/len(df)*100)

# Only represents 0.21% of awards

df[df['award_reference'].isna()]['funded_amount'].sum()/df['funded_amount'].sum()*100

# or 0 of total funded amount so we can neglect

0.20971854817568694


0.0

In [8]:
df_dropna = df[~df['award_reference'].isna()]

In [9]:
# award_reference is not Null and is duplicated
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference')['funded_amount'].sum()/df['funded_amount'].sum()*100

0.46772907392315577

In [10]:
df_dropna[df_dropna['award_reference'].duplicated(keep=False)].sort_values(by='award_reference').to_clipboard(index=False)

In [11]:
# Those duplicated represent 0.47% of total funded amount so we neglect

df_dropna.drop_duplicates(subset='award_reference', keep=False, inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\3630405099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna.drop_duplicates(subset='award_reference', keep=False, inplace=True)


In [12]:
df_dropna[df_dropna['start_date'].isna() & df_dropna['end_date'].isna()]['funded_amount'].sum() / df['funded_amount'].sum()*100

1.2092344278765894

In [13]:
# Awards where there are no start or end date (so we cannot tell if they are active or not) represent 1.21% of the total funded value

In [14]:
df_dropna = df_dropna[~(df_dropna['start_date'].isna() & df_dropna['end_date'].isna())]


In [15]:
df_dropna[df_dropna['start_date'].isna()]

,application_reference,award_reference,funder,administrator,application_source,announced_start_date,start_date,end_date,funded_amount,profiled_amount,project_cost,termination_date


In [16]:
df_dropna[df_dropna['end_date'].isna()]['funded_amount'].sum() # Those with end date is NaN are 0 funded amount so can neglect

0.0

In [17]:
df_dropna = df_dropna[~df_dropna['end_date'].isna()]

In [18]:
df_dropna[df_dropna['start_date'] > df_dropna['end_date']]['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

0.0010179126235217288

In [19]:
df_dropna = df_dropna[df_dropna['start_date'] < df_dropna['end_date']]


In [20]:
df_dropna['funded_amount'].sum() / df['funded_amount'].sum()*100

98.30318678376592

In [21]:
# So in total we can look at 98.2% of the total funded value, 1.8% is excluded

from datetime import date

today = date.today()
print("Today's date:", today)

# Active grants - start_date <= today and end_date >= today

data_ = df_dropna.copy()

data_['Active'] = 'Active'

data_.loc[data_.start_date > pd.to_datetime(today), 'Active'] = 'Future'

data_.loc[data_.end_date < pd.to_datetime(today), 'Active'] = 'Past'

Today's date: 2025-02-28


In [22]:
data_.drop(columns=['announced_start_date', 'profiled_amount', 'project_cost', 'termination_date'], inplace=True)

In [23]:
data_ # this is awards data

print(len(data_['funded_amount'])/len(df['funded_amount'])) # 98% of funded amount

0.9821643907437859


In [24]:
df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\108949258.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_application = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_application;", con)


In [25]:
dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\1554799385.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_opportunity = pd.read_sql("SELECT * FROM databank.dim_opportunity;", con)


In [26]:
dim_opportunity[['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource']].duplicated().sum()

1

In [27]:
dim_opportunity[dim_opportunity[['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource']].duplicated(keep=False)]

,OpportunitySKey,OpportunityName,OpportunityNumber,OpportunityStartDate,OpportunityEndDate,OpportunityCloseDate,OpportunityMode_AHRC,OpportunityOriginSource,OpportunityComments,ApplicantLedOrTargeted,IsResponsive
10091,3636717,Accelerating Innovation in Rail Round 5 (12 Mo...,61,2017-09-18 00:00:00,None,2017-11-15 12:00:00,None,IUK,None,Unassigned,Unassigned
10264,3636890,Accelerating Innovation in Rail Round 5 (12 Mo...,61,2017-09-18 00:00:00,None,2017-11-15 12:00:00,None,IUK,None,Unassigned,Unassigned


In [28]:
dim_opportunity.drop_duplicates(subset=['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource'], inplace=True)

In [29]:
data_merge = pd.merge(df_application, dim_opportunity, left_on=['opportunity_name', 'opportunity_id', 'application_source'],
                      right_on=['OpportunityName', 'OpportunityNumber', 'OpportunityOriginSource'], how='left', indicator=True)

In [30]:
data_merge.drop(columns=['_merge', 'application_status', 'derived_status', 'decision_date', 'lead_organisation_department_id', 'rejection_reason', 
                        'date_last_received', 'OpportunityStartDate', 'OpportunityEndDate', 'OpportunityCloseDate', 'international_programme', 'OpportunitySKey',
                        'OpportunityName', 'parent_award', 'parent_award_reference', 'redact_abstract_on_gtr', 'application_stage', 'application_stage_status',
                        'joint_proposal', 'OpportunityNumber', 'OpportunityOriginSource', 'publish_to_gtr'], inplace=True)

In [31]:

print(data_[['application_reference', 'application_source']].duplicated().sum())
# application_reference and application_source are duplicated

18


In [32]:
data_merge['award_reference'] = data_merge['award_reference'].replace('Unknown', None) # replace Unknown with NaN

In [33]:
len(data_[data_[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference'))

num = data_[data_[['application_reference', 'application_source']].duplicated(keep=False)].sort_values(by='application_reference')['funded_amount'].sum()

num / df['funded_amount'].sum()*100 # negligable

0.025595324812522236

In [34]:
data_.drop_duplicates(subset=['application_reference', 'application_source'], keep=False, inplace=True)

In [35]:
data_ # awards data

data_merge.drop(columns=['OpportunityComments', 'OpportunityMode_AHRC', 'scheme', 'full_economic_cost', 'application_title', 'decision_owner', 'administrator'],
                inplace=True)

In [36]:
data_merge # applications data with opportunity

,application_reference,award_reference,funder,application_source,opportunity_id,opportunity_name,application_category,lead_organisation_id,amount_applied_for,organisation_source,ApplicantLedOrTargeted,IsResponsive
0,1000,None,None,TFS,34,New Generation Thinkers 2022,Research and Innovation,2,NaN,TFS,Applicant-Led,Yes
1,10000,None,None,TFS,171,MRC Infections and immunity Sep 2023: responsi...,Research and Innovation,469,741068.0,TFS,Applicant-Led,Yes
2,10001,None,None,TFS,139,EPSRC open fellowship May 2023,Fellowship,34,1271665.6,TFS,Applicant-Led,Yes
3,10003,None,None,TFS,147,ESRC responsive mode: secondary data analysis ...,Research and Innovation,265,239954.4,TFS,Applicant-Led,Yes
4,1001,None,None,TFS,34,New Generation Thinkers 2022,Research and Innovation,17,NaN,TFS,Applicant-Led,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
582869,1214,CCF ST:London Social Venture Fund,RE,Offsystem,CCF ST,CCF RED,Research and Innovation Grant,10007775,1499500.0,Unknown,Unassigned,Unassigned
582870,1213,CCF ST:Innovation Catalysed & Enabled (ICE) Le...,RE,Offsystem,CCF ST,CCF RED,Research and Innovation Grant,10007796,329292.0,Unknown,Unassigned,Unassigned
582871,1212,CCF ST:AI SuperConnector,RE,Offsystem,CCF ST,CCF RED,Research and Innovation Grant,10003270,1444479.0,Unknown,Unassigned,Unassigned
582872,1211,CCF ST:North West Cyber Security Connect for C...,RE,Offsystem,CCF ST,CCF RED,Research and Innovation Grant,10007768,1230917.0,Unknown,Unassigned,Unassigned


In [37]:
data_w_opp = pd.merge(data_, data_merge.drop_duplicates(subset=['application_reference', 'application_source'], keep=False), 
                      on=['application_reference', 'application_source'], how='left', indicator=True)

In [38]:
data_w_opp[data_w_opp['_merge'] == 'left_only']['funded_amount'].sum() / df['funded_amount'].sum()*100 # negligable

0.0

In [39]:
print(len(data_w_opp))
print(len(data_))

206033
206033


In [40]:
data_w_opp.drop(columns=['administrator', 'award_reference_y', 'funder_y', 'amount_applied_for'], inplace=True)

In [41]:
data_w_opp['ApplicantLedOrTargeted'].value_counts() # 43% of awards are assigned as Applicant Led or Targetted - we can't say what the others are

ApplicantLedOrTargeted
Unassigned       87729
Applicant-Led    40501
Targeted         25052
Name: count, dtype: int64

In [42]:
87729/206033

0.4258007212436843

In [43]:
#data_grants = data_w_opp[(data_w_opp['start_date'].dt.year > 2015) & (data_w_opp['end_date'].dt.year < 2025)]
# select grants with start date 2019 - 2024

In [44]:
#data_grants['financial_year'] = data_grants['start_date'].dt.to_period('Q-MAR').dt.qyear

In [45]:
publications = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_outcome_publication;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\3013166775.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  publications = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_outcome_publication;", con)


In [46]:
publications.drop(columns=['isbn', 'issn_print', 'page_reference', 'publisher', 'chapter_number', 'conference', 'chapter_title',
                           'edition', 'issue', 'volume'], inplace=True)

In [47]:
print(len(publications[publications['award_reference'] == 'Unknown']))

print(len(publications))

209952/1643128

209952
1643128


0.12777580322409454

In [48]:
data_w_opp.rename(columns={'award_reference_x':'award_reference'}, inplace=True)

In [49]:
data_w_publications = pd.merge(
    data_w_opp[data_w_opp['application_source'] == 'Siebel'], 
    publications[publications['award_reference'] != 'Unknown'],
    on='award_reference'
)

In [50]:
print(len(data_w_publications[['award_reference', 'application_source']].drop_duplicates()))
print(len(data_w_opp))

56929/206033 # 28% of grants are linked to publications

56929
206033


0.2763101056626851

In [51]:
data_pub_count = data_w_publications.copy()[['start_date', 'end_date', 'year', 'funded_amount', 'Active', 'award_reference', 'ApplicantLedOrTargeted', 'doi', 'publication', 'application_category']]

In [52]:
data_pub_count['start_year'] = data_pub_count['start_date'].dt.year
data_pub_count['doi'].duplicated().sum()

609804

In [53]:
data_pub_count['year'] = data_pub_count['year'].replace('None/Unknown', None)
data_pub_count['year'] = data_pub_count['year'].replace('', None)
data_pub_count['year'] = data_pub_count['year'].replace('Pre-2006', None)


In [54]:
data_pub_count['year'].unique()

array(['2006', '2007', '2005', '2008', '2009', '2010', '2011', '2012',
       '2014', '2015', '2004', '2003', '2002', None, '2013', '2018',
       '2016', '2017', '2019', '2020', '2021', '2022', '2023', '2000',
       '0000', '2024', '1996', '1975', '1989', '1970', '1931', '1959',
       '2030', '1998', '1994', '1999', '1991', '1993', '1997', '1985',
       '1987', '1946', '2025', '1995', '1954', '2001', '1973', '1981',
       '1988', '1990', '1957', '1979', '1984', '1992', '1953', '1971',
       '1952', '1956', '1948', '1967', '1958', '1974', '1980', '1950',
       '1951', '1986', '1900', '1912', '1963', '1977', '1919', '1907',
       '1976', '1968', '1947', '1942', '1982', '1983', '1978', '1972'],
      dtype=object)

In [55]:

data_pub_count = data_pub_count.copy()[data_pub_count['start_year'] < data_pub_count['year'].astype(float)]


data_pub_count[(data_pub_count['start_year'] > 2017) & (data_pub_count['ApplicantLedOrTargeted'] == 'Applicant-Led') & (data_pub_count['application_category'] == 'Research Grant')].drop_duplicates(subset='doi').groupby(['year', 'start_year'], as_index=False).agg({'doi':'count'}).to_clipboard(index=False)


In [56]:
spinouts = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_outcome_spin_out;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\3447861269.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spinouts = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_outcome_spin_out;", con)


In [57]:
data_w_spinouts = pd.merge(
    data_w_opp[data_w_opp['application_source'] == 'Siebel'], 
    spinouts[spinouts['award_reference'] != 'Unknown'],
    on='award_reference'
)

In [58]:
select_spinouts = data_w_spinouts.copy()[['start_date', 'end_date', 'funded_amount', 'Active', 'award_reference', 'ApplicantLedOrTargeted', 'company', 'application_category', 'year_established', 'registration_number']]

In [60]:
select_spinouts['start_year'] = select_spinouts['start_date'].dt.year
select_spinouts['year_established'] = select_spinouts['year_established'].replace('Pre-2004', None)


select_spinouts = select_spinouts[select_spinouts['year_established'].astype(float) > select_spinouts['start_year']]

select_spinouts[(select_spinouts['ApplicantLedOrTargeted'] == 'Applicant-Led') & (select_spinouts['application_category'] == 'Research Grant')].drop_duplicates(subset='registration_number').groupby(['start_year', 'year_established'], as_index=False).agg({'registration_number':'count'}).to_clipboard(index=False)

In [61]:
print(len(select_spinouts)) # 2296


print(len(select_spinouts[select_spinouts['year_established'].astype(float) < select_spinouts['start_year']]))

1619
0


In [62]:
data_w_spinouts['year_established'].unique()

270/2296

0.11759581881533102

In [63]:
pp_table = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_project_partners;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_12120\3418696010.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pp_table = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_01_09_ssd_project_partners;", con)


In [65]:
pp_table['organisation_source'].unique()

array(['Siebel', 'Unknown', 'Research England'], dtype=object)

In [67]:
data_w_opp['application_source'].unique()

array(['TFS', 'Siebel', 'Offsystem', 'IUK - IFS', 'IUK - EU Funded',
       'IUK - Legacy Applications', 'IUK - SBRI', 'IUK - Legacy Smart',
       'IUK - Innovation Vouchers', 'IUK - KTP', 'Research England'],
      dtype=object)

In [70]:
# select application source is TFS Siebel and Offsystem, excluding IUK and RE
data_w_pp = pd.merge(
    data_w_opp[data_w_opp['application_source'].isin(['TFS', 'Siebel', 'Offsystem'])],
    pp_table, on='award_reference')

In [75]:
data_w_pp[data_w_pp['funder_x'] == 'RE']
data_w_pp[data_w_pp['application_source'] == 'Offsystem']

data_w_pp[(data_w_pp['application_category'] == 'Research Grant') & (data_w_pp['ApplicantLedOrTargeted'] == 'Applicant-Led')]

data_w_opp[data_w_opp['award_reference'].duplicated(keep=False)]

,application_reference,award_reference,funder_x,application_source,start_date,end_date,funded_amount,Active,opportunity_id,opportunity_name,application_category,lead_organisation_id,organisation_source,ApplicantLedOrTargeted,IsResponsive,_merge


In [94]:
data_w_pp['start_year'] = data_w_pp['start_date'].dt.year

data_w_pp[(data_w_pp['ApplicantLedOrTargeted'] == 'Applicant-Led') & (data_w_pp['application_category'] == 'Research Grant')].groupby('start_year', as_index=False).agg({'cash_value':'sum', 'in_kind_value':'sum'}).to_clipboard(index=False)


In [80]:
df[df['award_reference'].duplicated(keep=False)].dropna(subset='award_reference')

,application_reference,award_reference,funder,administrator,application_source,announced_start_date,start_date,end_date,funded_amount,profiled_amount,project_cost,termination_date
33,10883,209,EPSRC,EPSRC,TFS,2024-10-01,2024-06-01,2027-05-31,564065.61,0,NaN,None
34,10883,209,EPSRC,EPSRC,TFS,2024-10-01,2024-06-01,2027-05-31,548586.57,0,NaN,None
87,11799,95,EPSRC,EPSRC,TFS,2024-10-01,2024-05-01,2027-04-30,182026.09,0,NaN,None
146,12917,150,EPSRC,EPSRC,TFS,2025-06-01,2024-06-01,2026-09-30,531560.76,0,NaN,None
147,12917,150,EPSRC,EPSRC,TFS,2025-06-01,2024-06-01,2026-09-30,531965.07,0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
209346,434,434,SiPF,RE,Research England,None,2019-06-01,2019-08-31,50000.00,0,NaN,None
209370,490,490,RE,RE,Research England,None,2019-08-01,2020-07-31,885405.00,0,NaN,None
209371,491,491,RE,RE,Research England,None,2019-08-01,2020-07-31,1493000.00,0,NaN,None
209382,503,503,RE,RE,Research England,None,2019-08-01,2020-07-30,542453.00,0,NaN,None


In [89]:
pd.merge(
    df[df['award_reference'].duplicated(keep=False)].dropna(subset='award_reference').drop_duplicates(subset=['award_reference', 'application_source'], keep=False),
    pp_table,
    on='award_reference').sort_values(by='award_reference').query('application_source != "Research England"')



,application_reference_x,award_reference,funder,administrator,application_source,announced_start_date,start_date,end_date,funded_amount,profiled_amount,project_cost,termination_date,application_reference_y,project_partner_organisation_id,cash_value,in_kind_value,organisation_source
5,2449,10,BBSRC,BBSRC,TFS,2023-07-01,2023-07-01,2025-06-30,207395.24,0,NaN,None,10,Unknown,20130000.00,19260000.0,Unknown
8,34827,111,ESRC,ESRC,TFS,2024-07-01,2024-07-01,2025-06-30,171345.72,0,NaN,None,111,Unknown,18608000.00,2347000.0,Unknown
2,5972,120,EPSRC,EPSRC,TFS,2024-10-01,2024-05-01,2027-04-30,461600.44,0,NaN,None,120,Unknown,3988900.00,26978000.0,Unknown
1,17787,130,EPSRC,EPSRC,TFS,2024-10-01,2024-08-01,2027-01-31,390944.92,0,NaN,None,130,Unknown,52754000.00,NaN,Unknown
11,37837,172,EPSRC,EPSRC,TFS,2024-10-01,2024-10-01,2025-09-30,75826.77,0,NaN,None,172,Unknown,10006942.00,10873000.0,Unknown
16,46969,228,Unknown,MRC,TFS,2025-04-01,2025-04-01,2026-04-30,242492.28,0,NaN,None,228,Unknown,3417717.00,NaN,Unknown
17,48606,233,Unknown,MRC,TFS,2024-11-01,2024-11-01,2025-10-31,244382.00,0,NaN,None,233,Unknown,3085341.00,NaN,Unknown
9,35401,240,NERC,NERC,TFS,2024-10-01,2024-07-01,2028-06-30,4761440.31,0,NaN,None,240,Unknown,355134.00,NaN,Unknown
10,29458,253,EPSRC,EPSRC,TFS,2024-09-01,2024-09-01,2025-08-31,75387.56,0,NaN,None,253,Unknown,227000.00,NaN,Unknown
19,45013,269,Unknown,AHRC,TFS,2025-02-01,2025-02-01,2026-01-31,102034.82,0,NaN,None,269,Unknown,2945000.00,NaN,Unknown


In [87]:
df[df['award_reference'] == '277']
df[df['award_reference'] == 'EP/Z536052/1']

,application_reference,award_reference,funder,administrator,application_source,announced_start_date,start_date,end_date,funded_amount,profiled_amount,project_cost,termination_date
1491,30244,EP/Z536052/1,Unknown,EPSRC,TFS,None,2025-01-01,2028-12-31,5216140.37,0,NaN,None
2381,30244,EP/Z536052/1,Unknown,EPSRC,TFS,2025-01-01,2025-01-01,2028-12-31,5216140.37,0,NaN,None
